In [1]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import ast
import os
import random
import re
import string
import warnings
from functools import partial
from itertools import islice, product
from typing import Tuple

import nltk
import numpy as np
import optuna
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import wget
from corus import load_lenta
from gensim.corpora import Dictionary
from gensim.models import Phrases
from gensim.models.ldamodel import LdaModel
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from num2words import num2words
from optuna.samplers import TPESampler
from pymystem3 import Mystem
from tqdm import tqdm

pyLDAvis.enable_notebook()
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     /home/starminalush/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/starminalush/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
RANDOM_SEED = 42

In [4]:
random.seed(RANDOM_SEED)
os.environ["PYTHONHASHSEED"] = str(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [5]:
morph_analyzer = Mystem()
tokenizer = WordPunctTokenizer()

In [6]:
russian_stopwords = stopwords.words("russian")
additional_sw = "мои оно мной мною мог могут мор мое мочь оба нам нами ними однако нему никуда наш нею неё наша наше наши очень отсюда вон вами ваш ваша ваше ваши весь всем всеми вся ими ею будем будете будешь буду будь будут кому кого которой которого которая которые который которых кем каждое каждая каждые каждый кажется та те тому собой тобой собою тобою тою хотеть хочешь свое свои твой своей своего своих твоя твоё сама сами теми само самом самому самой самого самим самими самих саму чему тебе такое такие также такая сих тех ту эта это этому туда этим этими этих абы аж ан благо буде вроде дабы едва ежели затем зато ибо итак кабы коли коль либо лишь нежели пока покамест покуда поскольку притом причем пускай пусть ровно сиречь словно также точно хотя чисто якобы "
pronouns = "я мы ты вы он она оно они себя мой твой ваш наш свой его ее их то это тот этот такой таков столько весь всякий сам самый каждый любой иной другой кто что какой каков чей сколько никто ничто некого нечего никакой ничей нисколько кто-то кое-кто кто-нибудь кто-либо что-то кое-что что-нибудь что-либо какой-то какой-либо какой-нибудь некто нечто некоторый некий"
conjunctions = "что чтобы как когда ибо пока будто словно если потому что оттого что так как так что лишь только как будто с тех пор как в связи с тем что для того чтобы кто как когда который какой где куда откуда"
digits = "ноль один два три четыре пять шесть семь восемь девять десять одиннадцать двенадцать тринадцать четырнадцать пятнадцать шестнадцать семнадцать восемнадцать девятнадцать двадцать тридцать сорок пятьдесят шестьдесят семьдесят восемьдесят девяносто сто"
modal_words = "вероятно возможно видимо по-видимому кажется наверное безусловно верно  действительно конечно несомненно разумеется"
particles = "да так точно ну да не ни неужели ли разве а что ли что за то-то как ну и ведь даже еще ведь уже все все-таки просто прямо вон это вот как словно будто точно как будто вроде как бы именно как раз подлинно ровно лишь только хоть всего исключительно вряд ли едва ли"
prepositions = "близ  вблизи  вдоль  вокруг  впереди  внутрь  внутри  возле  около  поверх  сверху  сверх  позади  сзади  сквозь  среди  прежде  мимо  вслед  согласно  подобно  навстречу  против  напротив  вопреки  после  кроме  вместе  вдали  наряду  совместно  согласно  нежели вроде от бишь до без аж тех раньше совсем только итак например из прямо ли следствие а поскольку благо пускай благодаря случае затем притом также связи время при чтоб просто того невзирая даром вместо точно покуда тогда зато ради ан буде прежде насчет раз причине тому так даже исходя коль кабы более ровно либо помимо как-то будто если словно лишь бы и не будь пор тоже разве чуть как хотя наряду потому пусть в равно между сверх ибо на судя то чтобы относительно или счет за но сравнению причем оттого есть когда уж ввиду тем для дабы чем хоть с вплоть скоро едва после той да вопреки ежели кроме сиречь же коли под абы несмотря все пока покамест паче прямо-таки перед что по вдруг якобы подобно"
evaluative = "наиболее наименее лучший больший высший низший худший более менее"

russian_stopwords.extend(additional_sw.split())
russian_stopwords.extend(pronouns.split())
russian_stopwords.extend(conjunctions.split())
russian_stopwords.extend(digits.split())
russian_stopwords.extend(modal_words.split())
russian_stopwords.extend(particles.split())
russian_stopwords.extend(prepositions.split())
russian_stopwords.extend(evaluative.split())
russian_stopwords = set(russian_stopwords)

## Скачиваем датасет, делаем предобработку

In [7]:
dataset_url = "https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz"
wget.download(dataset_url)

100% [..................................................] 527373240 / 527373240

'lenta-ru-news.csv.gz'

In [8]:
path = "lenta-ru-news.csv.gz"
records = load_lenta(path)

Датасет большой, поэтому возьмем 3000 записей

In [9]:
news = (record.title + ". " + record.text for record in islice(records, 3000))

### Делаем препроцессинг текста

In [10]:
def convert_digit_to_word(text):
    pattern = r"\d+"
    numbers = re.findall(pattern, text)
    for number in numbers:
        text = text.replace(number, num2words(number, lang="ru"))
    return text


def lemmatize(words) -> list[str]:
    lemmatized_text = morph_analyzer.lemmatize(" ".join(words))
    return [word for word in lemmatized_text if word.isalnum()]


def delete_stop_words(words):
    return [word for word in words if word not in russian_stopwords]


def delete_punctuation(text):
    return " ".join([word for word in tokenizer.tokenize(text) if word.isalnum()])


def tokenize_by_sentences(text: str) -> list[str]:
    return sent_tokenize(text, language="russian")


def tokenize_by_words(text):
    return word_tokenize(text, language="russian")


def convert_text_to_lowercase(text: str):
    return text.lower()


def preprocess(text: str) -> list[str]:
    sentences: list[str] = tokenize_by_sentences(text)
    preprocessed_sentences: list[str] = list()
    for sentence in sentences:
        sentence = convert_text_to_lowercase(sentence)
        sentence = convert_digit_to_word(sentence)
        sentence = delete_punctuation(sentence)
        words = tokenize_by_words(sentence)
        words = delete_stop_words(words)
        words = lemmatize(words)
        preprocessed_sentences.extend(words)
    return preprocessed_sentences

In [11]:
preprocessed_news = [preprocess(record) for record in tqdm(news)]

3000it [00:31, 93.97it/s] 


## Обучение LDA

Подготовим словарь и корпус для LDA из датасета

In [12]:
bigram = Phrases(preprocessed_news, min_count=5, threshold=100)
bigram_documents = [bigram[doc] for doc in preprocessed_news]

dictionary = Dictionary(bigram_documents)
corpus = [dictionary.doc2bow(doc) for doc in bigram_documents]

Начинаем эксперименты по подбору параметров

In [13]:
def train(
    num_topics: int,
    alpha: float,
    eta: float,
    chunk_size: int,
    passes: int,
    dictionary: Dictionary,
    corpus,
):
    lda_model = LdaModel(
        corpus,
        num_topics=num_topics,
        id2word=dictionary,
        random_state=RANDOM_SEED,
        update_every=1,
        alpha=alpha,
        eta=eta,
        chunksize=chunk_size,
        passes=passes,
        iterations=150,
        per_word_topics=True,
    )

    perplexity = lda_model.log_perplexity(corpus)
    print(
        f"LDA с параметрами num_topics {num_topics}, alpha {alpha},  eta {eta}, chunksize {chunk_size} и passes {passes} -  perplexity: {perplexity}"
    )
    return perplexity, lda_model

In [14]:
def objective(trial, corpus, dictionary, texts):
    num_topics = trial.suggest_int("num_topics", 5, 25)
    alpha = trial.suggest_float("alpha", 0.01, 1.0)
    eta = trial.suggest_float("eta", 0.01, 1.0)
    passes = trial.suggest_int("passes", 1, 5)
    chunk_size = trial.suggest_int("chunk_size", 5, 30)
    perplexity, _ = train(
        num_topics=num_topics,
        alpha=alpha,
        eta=eta,
        chunk_size=chunk_size,
        passes=passes,
        dictionary=dictionary,
        corpus=corpus,
    )
    return perplexity

In [15]:
%%time
sampler = TPESampler(seed=RANDOM_SEED)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(
    partial(objective, corpus=corpus, dictionary=dictionary, texts=preprocessed_news),
    n_trials=15,
    show_progress_bar=True,
)

best_trial = study.best_trial
best_params = study.best_params

[I 2023-11-12 01:28:53,886] A new study created in memory with name: no-name-5585bede-2816-4709-99c2-498c90defebd


  0%|          | 0/15 [00:00<?, ?it/s]

LDA с параметрами num_topics 12, alpha 0.951207163345817,  eta 0.7346740023932911, chunksize 9 и passes 3 -  perplexity: -9.21416983268644
[I 2023-11-12 01:29:36,899] Trial 0 finished with value: -9.21416983268644 and parameters: {'num_topics': 12, 'alpha': 0.951207163345817, 'eta': 0.7346740023932911, 'passes': 3, 'chunk_size': 9}. Best is trial 0 with value: -9.21416983268644.
LDA с параметрами num_topics 8, alpha 0.06750277604651747,  eta 0.8675143843171859, chunksize 23 и passes 4 -  perplexity: -8.985171500724398
[I 2023-11-12 01:29:58,797] Trial 1 finished with value: -8.985171500724398 and parameters: {'num_topics': 8, 'alpha': 0.06750277604651747, 'eta': 0.8675143843171859, 'passes': 4, 'chunk_size': 23}. Best is trial 1 with value: -8.985171500724398.
LDA с параметрами num_topics 5, alpha 0.9702107536403743,  eta 0.8341182143924175, chunksize 9 и passes 2 -  perplexity: -9.095776969448936
[I 2023-11-12 01:30:16,387] Trial 2 finished with value: -9.095776969448936 and parameter

Обучим модель с лучшими параметрами

In [16]:
_, model = train(**best_params, dictionary=dictionary, corpus=corpus)

LDA с параметрами num_topics 10, alpha 0.020716482880900333,  eta 0.7699497377237832, chunksize 21 и passes 4 -  perplexity: -8.971536038622316


In [17]:
def visualize_model(model: LdaModel, corpus: list[list[Tuple[int]]]):
    return gensimvis.prepare(model, corpus, dictionary=model.id2word, mds="mmds")

### Визуализируем самую лучшую модель

In [18]:
graph = visualize_model(model=model, corpus=corpus)

/home/starminalush/study/itmo/dl_and_nlp_itmo_2023/venv/lib/python3.10/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [20]:
pyLDAvis.save_html(graph, "lda.html")

In [22]:
pyLDAvis.display(graph)

Посмотрим на топики модели

In [21]:
model.print_topics()

[(0,
  '0.004*"фильм" + 0.004*"концерт" + 0.003*"хаска" + 0.003*"рэпер" + 0.003*"артист" + 0.003*"лента" + 0.002*"музыкант" + 0.002*"певец" + 0.002*"режиссер" + 0.002*"песня"'),
 (1,
  '0.025*"украина" + 0.021*"украинский" + 0.015*"корабль" + 0.012*"российский" + 0.011*"россия" + 0.009*"керченский_пролив" + 0.009*"фсб" + 0.009*"ноябрь" + 0.008*"киев" + 0.008*"военный_положение"'),
 (2,
  '0.007*"мужчина" + 0.006*"женщина" + 0.006*"ребенок" + 0.005*"летний" + 0.005*"человек" + 0.004*"сообщать" + 0.004*"полиция" + 0.003*"видео" + 0.003*"рассказывать" + 0.003*"обнаруживать"'),
 (3,
  '0.016*"тысяча" + 0.014*"год" + 0.012*"два" + 0.010*"россия" + 0.008*"ноябрь" + 0.007*"сообщать" + 0.006*"слово" + 0.006*"российский" + 0.005*"заявлять" + 0.004*"страна"'),
 (4,
  '0.005*"самолет" + 0.005*"ученый" + 0.003*"полет" + 0.003*"роскосмос" + 0.003*"ракета" + 0.002*"система" + 0.002*"луна" + 0.002*"спутник" + 0.002*"f" + 0.002*"море"'),
 (5,
  '0.002*"браудер" + 0.002*"заключенный" + 0.001*"осужденны

Вывод: визуально модель делит текст на топики приемлимо